In [212]:
%matplotlib notebook
import matplotlib as mpl
mpl.rcParams['figure.figsize'] = [8, 8]
import matplotlib.pyplot as plt

import shapely
import itertools
import pandas as pd
import geopandas as gpd
from faker import Factory

fake = Factory.create()

In [35]:
class Box(object):
    def __init__(self, borigin: tuple, bsize: float):
        self.borigin = borigin
        self.bsize = bsize
        
        self.xmin = self.borigin[0]
        self.xmax = self.borigin[0] + self.bsize
        self.ymin = self.borigin[1]
        self.ymax = self.borigin[1] + self.bsize
        
    def makebox(self):
        self.boxgeom = shapely.geometry.box(self.xmin, self.ymin, self.xmax, self.ymax)
        self.boxgeom_wkt = self.boxgeom.wkt
        self.centroid_wkt = self.boxgeom.centroid.wkt

In [198]:
class makeGrid(object):
    def __init__(self, samples, gorigin: tuple, cellsize: float, nrows_ncols: tuple, Box, gridname = ''):
        
        self.samples = samples
        assert isinstance(self.samples, gpd.GeoDataFrame)
        self.nrows = nrows_ncols[0]
        self.ncols = nrows_ncols[1]
        
        self.cellsize = cellsize
        
        self.gridname = gridname
        self.gorigin = gorigin
        self.gorigin_x = self.gorigin[0]
        self.gorigin_y = self.gorigin[1]
        
        self.gorigin_x = self.gorigin_x - self.cellsize
        self.gorigin_y = self.gorigin_y - self.cellsize
        
        print(f'Grid cellsize: {self.cellsize}, rows: {self.nrows}, cols: {self.ncols}, Origin: {self.gorigin_x} / {self.gorigin_y}')

    def make_rows(self):
        self.rows = []
        for row in range(0, self.nrows):
            self.gorigin_x = self.gorigin_x + self.cellsize
            self.rows.append(self.gorigin_x)
    
    def make_cols(self):
        self.cols = []
        for row in range(0, self.ncols):
            self.gorigin_y = self.gorigin_y + self.cellsize
            self.cols.append(self.gorigin_y)
            
    def make_grid(self):
        
        self.make_rows()
        self.make_cols()
        
        grid_origins = list(itertools.product(self.rows, self.cols))
        
        boxes = []
        colours = []
        
        for i in grid_origins:
            colour = fake.hex_color()
            x = Box(i, self.cellsize)
            x.makebox()
            boxes.append(x.boxgeom)
            colours.append(colour)
        
        grid_origins_dict = {'geometry':  boxes, 'colour': colours}
        
        self.boxes_gdf = gpd.GeoDataFrame(grid_origins_dict, crs={'init' :'epsg:4283'})
        self.boxes_gdf = self.boxes_gdf.set_geometry('geometry')
        
    def export_geojson(self):
        self.boxes_gdf.to_file(f'{self.gridname}_boxes.geojson', driver = 'GeoJSON')
        

In [199]:
# Unpickle some points to test
import pickle

with open("broken_hill_surface_samples_normalised_7.pickle", 'rb') as f:
    samples = pickle.load(f)

In [200]:
_gdf = gpd.GeoDataFrame(samples, crs={'init' :'epsg:4283'})

sample_locs = _gdf[['RIN', 'SAMPCODE', 'index', 'LAT94', 'LNG94', 'Au_ppm', 'normalised', 'classifications', 'stroke']]

sample_locs['normalised'].fillna(0, inplace=True)
sample_locs['classifications'].replace([0], '#82817d', inplace=True)

sample_locs['Coords'] = list(zip(sample_locs.LNG94, sample_locs.LAT94))
sample_locs['Coords'] = sample_locs['Coords'].apply(shapely.geometry.Point)
sample_locs = sample_locs.set_geometry('Coords')
sample_locs.crs = {'init' :'epsg:4283'}

C:\Users\gates\Anaconda3\lib\site-packages\pandas\core\generic.py:5430: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)
C:\Users\gates\Anaconda3\lib\site-packages\pandas\core\generic.py:5886: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)
C:\Users\gates\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\gates\Anaco

In [201]:
def cell_rows_cols(samples, cellsize):

    samples.total_bounds
    
    minx, miny, maxx, maxy = sample_locs.total_bounds
    
    xmin = minx - (0.5*cellsize)
    ymin = miny - (0.5*cellsize)
    
    width = maxx - minx
    height = maxy - miny

    nrows = abs(width) / cellsize
    ncols = abs(height) / cellsize

    nrows = int(nrows.round()) 
    ncols = int(ncols.round())
    
    return [(xmin, ymin), (nrows +1, ncols +1)]

In [202]:
def grid_samples(samples, grid_gdf):
    
    grid_with_samples = gpd.sjoin(samples, grid_gdf, how="inner", op='intersects')
    
    max_vals = grid_with_samples.groupby(['index_right']).agg({'normalised': max})
        
    max_values_grids = grid.boxes_gdf.merge(max_vals, on='index_right')
    
    return max_values_grids

def grid_geojson(max_values_grids, name):
    max_values_grids.to_file(f'{name}.geojson', driver = 'GeoJSON')
    print('Exported')

In [203]:
cellsize = 0.05

grid_params = cell_rows_cols(sample_locs, cellsize)

In [204]:
grid_params[1]

(13, 21)

In [205]:
# def __init__(self, samples, gorigin: tuple, cellsize: float, nrows_ncols: tuple, Box, gridname = ''):

grid_name = 'testing_grid_new'

grid = makeGrid(sample_locs, grid_params[0], cellsize, grid_params[1], Box, grid_name)
grid.make_grid()

Grid cellsize: 0.05, rows: 13, cols: 21, Origin: 140.92669791804084 / -32.29666600562881


In [206]:
max_values_grids = grid_samples(sample_locs, grid.boxes_gdf)
grid_geojson(max_values_grids, grid_name)

Exported


In [214]:
sample_locs

,RIN,SAMPCODE,index,LAT94,LNG94,Au_ppm,normalised,classifications,stroke,Coords
index_left,,,,,,,,,,
0,RE0008188,SOIL,0,-31.237320,141.473279,0.00258,0.768924,#e6a94e,#1fdff9,POINT (141.4732785908537 -31.23732028765827)
1,RE0008188,SOIL,0,-31.237319,141.473699,0.00104,0.054770,#82817d,#1fdff9,POINT (141.4736986173678 -31.23731874095523)
2,RE0008188,SOIL,0,-31.237317,141.474119,0.00158,0.383487,#55b1d9,#1fdff9,POINT (141.4741186438495 -31.23731719288023)
3,RE0008188,SOIL,0,-31.237316,141.474539,0.00316,0.928316,#e02d2d,#1fdff9,POINT (141.4745386702988 -31.23731564343327)
4,RE0008188,SOIL,0,-31.237314,141.474959,0.00142,0.299565,#55b1d9,#1fdff9,POINT (141.4749586967156 -31.23731409261435)
5,RE0008188,SOIL,0,-31.237313,141.475379,0.00190,0.528452,#55b1d9,#1fdff9,POINT (141.4753787230999 -31.23731254042346)
6,RE0008188,SOIL,0,-31.237311,141.475799,0.00127,0.211814,#82817d,#1fdff9,POINT (141.4757987494517 -31.23731098686061)
7,RE0008188,SOIL,0,-31.237309,141.476219,0.00241,0.715347,#5bd955,#1fdff9,POINT (141.476218775771 -31.23730943192579)
8,RE0008188,SOIL,0,-31.237308,141.476639,0.00190,0.528452,#55b1d9,#1fdff9,POINT (141.4766388020577 -31.23730787561902)


In [207]:
# Show me the grid and the points!!!!
import folium

m = folium.Map(location=[-32, 141.5], zoom_start=9)

# ADD AS POINTS
samples = folium.FeatureGroup(name="samples")
sample_points = list(zip(sample_locs.LAT94.values, sample_locs.LNG94.values, sample_locs.normalised.values, sample_locs.classifications.values))
for lat, lng, au, colour in sample_points:
    samples.add_child(folium.CircleMarker(location=[lat, lng ], radius=1,
        popup=str(au), 
        tooltip=str(au),
        fill=True,  # Set fill to True
        color=str(colour),
        fill_opacity=0.5)).add_to(m)

# # ADD AS GEOJSON
# folium.GeoJson('samples_loc.geojson',
#     style_function=lambda x: {
#         'color' : 'grey',
#         'weight' : 1,
#         'opacity': 1,
#         'fillColor' : x['properties']['stroke'],
#         }).add_to(m)

folium.GeoJson(f'{grid_name}.geojson',
    style_function=lambda x: {
        'color' : 'grey',
        'weight' : 2,
        'opacity': 0.2,
        'fillColor' : x['properties']['colour'],
        }).add_to(m)

m.add_child(folium.LayerControl())
m.save(f"{grid_name}.html")

In [ ]:
pd.merge(grid.boxes_gdf, grid_with_samples, how='inner', on=None, left_on=None, right_on=None,
         left_index=False, right_index=False, sort=True,
         suffixes=('_x', '_y'), copy=True, indicator=False,
         validate=None)

In [81]:
for idx, samples in grid_with_samples.groupby('index_right'):
    print(samples['normalised'].max())
    print(samples['normalised'].std())
    print(samples['normalised'].count())

2.375535566118861
0.36131011743047725
3270
2.4620903319182545
0.3328782333505457
766
0.0
nan
1
1.0342125383512786
0.3233149209769976
150
0.0
0.0
63
0.0
0.0
98
2.3322353158840556
0.3551205846510033
2555


In [63]:
_df = pd.DataFrame(grid.boxes_gdf)
# _df['index'] = _df.index
_dfgrp = _df.groupby('colour')

for i, d in _dfgrp:
    
    _df = pd.DataFrame(d, columns = {'geometry': 'geometry', 'index': 'index', 'colour': 'colour'})
    _gdf = gpd.GeoDataFrame()
    _gdf.crs = {'init' :'epsg:4326'}
    _gdf = gpd.GeoDataFrame(_df)
    _gdf = _gdf.set_geometry('geometry')
    
#     _grid_with_samples = gpd.sjoin(sample_locs, _gdf, how="inner", op='intersects')

In [50]:
# sample_locs.to_file('samples_loc.geojson', driver ='GeoJSON')
# grid.export_geojson()

In [64]:
_gdf

,geometry,index,colour
index_right,,,
11,POLYGON ((141.6016979180408 -32.02166600562881...,11,#f77968


In [ ]:
# Show me the grid and the points!!!!
import folium

m = folium.Map(location=[-32, 141.5], zoom_start=9)

# ADD AS POINTS
# samples = folium.FeatureGroup(name="samples")
# sample_points = list(zip(sample_locs.LAT94.values, sample_locs.LNG94.values, sample_locs.normalised.values, sample_locs.classifications.values))
# for lat, lng, au, colour in sample_points:
#     samples.add_child(folium.CircleMarker(location=[lat, lng ], radius=1,
#         popup=str(au), 
#         tooltip=str(au),
#         fill=True,  # Set fill to True
#         color=str(colour),
#         fill_opacity=0.5)).add_to(m)

# ADD AS GEOJSON
folium.GeoJson('samples_loc.geojson',
    style_function=lambda x: {
        'color' : 'grey',
        'weight' : 1,
        'opacity': 1,
        'fillColor' : x['properties']['stroke'],
        }).add_to(m)

folium.GeoJson('grid_points_boxes.geojson',
    style_function=lambda x: {
        'color' : 'grey',
        'weight' : 2,
        'opacity': 0.2,
        'fillColor' : x['properties']['colour'],
        }).add_to(m)

m.add_child(folium.LayerControl())
m.save("grid_with_points_boxes.html")

# IFrame(src='ALL_SAMPLES.html', width=1000, height=600)

In [21]:
list(itertools.product([0,1,2], [1,2,3,4,5]))

[(0, 1),
 (0, 2),
 (0, 3),
 (0, 4),
 (0, 5),
 (1, 1),
 (1, 2),
 (1, 3),
 (1, 4),
 (1, 5),
 (2, 1),
 (2, 2),
 (2, 3),
 (2, 4),
 (2, 5)]